In [1]:
import os

## NEW
- skip threading and FastRelax

In [2]:
array_number=240 # If sequence scaffolds, then it should product of number 7
array_limit=6

scaffold_dir="../../scaffolds/scaffolds_V4/"
separate_scaffold_folders=True

prefix="pcsk9"
target_pdb="examples/pcsk9_target_short.pdb"
hotspots='ppi.hotspot_res=[A59,A61,A64,A66]'

num_of_diffusions=20 # Number of RF diffusions per script
total_mpnn=50 # Total mpnn sequences for per design
mpnn_per_design=50 # Filtered mpnn sequences for AF2

# Other parameters
num_recycles=3 # AF2 recycles
sampling_temp=0.0001 # ProteinMPNN sampling temperature
print(f"This will generate {num_of_diffusions*mpnn_per_design} designs per array job! In total {num_of_diffusions*mpnn_per_design*array_number} designs ({num_of_diffusions*array_number} diffusions)!")

This will generate 1000 designs per array job! In total 240000 designs (4800 diffusions)!


In [3]:
def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subdirectories.append(entry.path)
    return subdirectories

if separate_scaffold_folders:
    scaffold_list=get_subdirectories(scaffold_dir)
    scaffolds=len(scaffold_list)
    print(f"Script will run {scaffolds} array scripts for each scaffold with {array_number//scaffolds} jobs ({num_of_diffusions*mpnn_per_design} designs each) with total of {(array_number/scaffolds)*(num_of_diffusions*mpnn_per_design)} sampled designes per scaffold")

Script will run 6 array scripts for each scaffold with 40 jobs (1000 designs each) with total of 40000.0 sampled designes per scaffold


In [4]:
scaffold_list

['../../scaffolds/scaffolds_V4/27_3H',
 '../../scaffolds/scaffolds_V4/60_3Hs',
 '../../scaffolds/scaffolds_V4/4H_9',
 '../../scaffolds/scaffolds_V4/51_3H',
 '../../scaffolds/scaffolds_V4/lcb3',
 '../../scaffolds/scaffolds_V4/2H3E']

## Runs slurm array script

In [5]:
import subprocess

if separate_scaffold_folders:
    array_number=array_number//scaffolds
    array_limit=array_limit//scaffolds

    for i in range(scaffolds):
        # Setup arguments
        scaffold_dir=scaffold_list[i]
        script_arguments=f"{num_of_diffusions} {total_mpnn} {mpnn_per_design} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
        bash_arguments=f"--job-name={prefix}_binder_docking --array=1-{array_number}%{array_limit}"
        
        command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

        # Run the array bash script
        print(f"Running {command}")
        subprocess.run(command, shell=True)

else:
    script_arguments=f"{num_of_diffusions} {total_mpnn} {mpnn_per_design} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
    bash_arguments=f"--job-name={prefix}_binder_docking --array=1-{array_number}%{array_limit}"
        
    command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

    # Run the array bash script
    print(f"Running {command}")
    subprocess.run(command, shell=True)

#print(f"command: {command}")

Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/27_3H AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328456
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/60_3Hs AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328457
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/4H_9 AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328458
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/b

In [13]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
   328447_[8-56%8]       gpu predict_  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328456_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328457_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328458_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328459_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328460_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328461_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
          328447_0       gpu predict_  tsatler  R       2:08      1 compute-0-12
          328447_1       gpu predict_  tsatler  R       2:08      1 compute-0-12
          328447_2       gpu predict_  tsatler  R       2:08      1 compute-6-1
          328447_3       gpu predict_  tsatler  R       2